In [10]:
#Get the annotations
import os
annotations = [os.path.join(r'/root/Downloads/Dataset/Labels', x) for x in os.listdir(r'/root/Downloads/Dataset/Labels') if x[-3:] == "xml"]# annotations.sort()
# print(annotations)

In [4]:
path ="/root/Downloads/IDD_Detection/JPEGImages"
i=0
for j in os.listdir(path):
    path1= path+'/'+j
    for k in os.listdir(path1):
        path2=path1+'/'+k
        for l in os.listdir(path2):
            os.rename(path2+'/'+l, "/root/Downloads/Dataset/Images"+'/'+j+k+l.split('.')[0]+'.jpg')

In [6]:
path ="/root/Downloads/IDD_Detection/Annotations"
i=0
for j in os.listdir(path):
    path1= path+'/'+j
    for k in os.listdir(path1):
        path2=path1+'/'+k
        for l in os.listdir(path2):
            if ".swp" in l:
                pass
            else:
                os.rename(path2+'/'+l, "/root/Downloads/Dataset/Labels"+'/'+j+k+l.split('.')[0]+'.xml')


In [12]:
# %pip install scikit-learn
# %pip install tqdm
# %pip install matplotlib

In [11]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
# import matplotlib.pyplot as plt

random.seed(108)
# Function to get the data from XML Annotation

def extract_info_from_xml(xml_file):
    f_name = xml_file.split("/")[-1]
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = f_name#elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict
# Dictionary that maps class names to IDs
# class_name_to_id_mapping = {"trafficlight": 0,
#                            "stop": 1,
#                            "speedlimit": 2,
#                            "crosswalk": 3}

class_name_to_id_mapping = {"bicycle": 0,
                           "bus": 1,
                           "traffic sign": 2,
                           "train": 3,
                           "motorcycle": 4,
                           "car": 5,
                           "traffic light": 6,
                           "person": 7,
                           "vehicle fallback": 8,
                           "truck": 9,
                           "autorickshaw": 10,
                           "animal": 11,
                           "caravan": 12,
                           "rider": 13,
                           "trailer": 14                       
                            }


# Convert the info dict to the required yolo format and write it to disk

def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    # save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
    save_file_name = os.path.join(r"/root/Downloads/Dataset/Labels_XML", info_dict["filename"].replace("xml", "txt"))
#     save_file_name = info_dict["filename"].replace("jpg", "txt")
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [9]:
annotations

[]

In [12]:

for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 41857/41857 [00:07<00:00, 5548.80it/s]
